# Run regenie: Resistant hypertension replication

This script preprocesses genotype data and runs GWAS with SNP data. We replicate analysis performed earlier with FinnGen data. Both analysis steps will be carried out with SNP data as was done in FinnGen.

Phenotypes have been created by script **create_ukbb_pheno_resht.Rmd**

**Variable definitions**

* Genotypes: 
  - 22418, chip data (GRCh37) for regenie step 1
  - 21008, imputed data, Genomics England (GRCh38) for regenie step2
* Phenotypes: 
  - resht_fg: Number of hypertensive medication classes $\geq$ 4 and hypertension
  - resht_ukb: Number of medication classes $\geq$ 3, Systolic BP  $\geq$ 140 or diastolic BP  $\geq$ 90 and hypertension
  - resht_both: resht_fg or resht_ukbb
  - Control for all variables: One hypertensive medication class and hypertension
 * Covariates:
   - 21003, Age when attended assessment centre (Baseline characteristics)
   - 22001, Sex (Genotypes)
   - 22009, Genetic principal components (Genotypes)
   
**Run variables**

In [1]:
name="resht"
pheno_file_dir="/data"
geno_file_dir="/data/genotype"
regenie_out_dir="/data/regenie"

geno_var="ukb22418"
geno_in="/mnt/project/data/lifted_geno_in/${geno_var}_c*.???"  #[1-9]*"
bed_base="${geno_var}_c1_22_merged" #_c10_b0_v2_GRCh38_full_analysis_set_plus_decoy_hla.fam  

geno2_var="ukb21008"
geno2_indir="/Bulk/Imputation/Imputation\ from\ genotype\ \(GEL\)"

pheno_in="resht_pheno.tsv"
pheno_cols="resht_fg,resht_ukb,resht_both"
covar_cols="p21003,p22001,p22009_a1,p22009_a2,p22009_a3,p22009_a4,p22009_a5,p22009_a6,p22009_a7,p22009_a8,p22009_a9,p22009_a10"

instance="mem1_ssd1_v2_x16"

## Lift over
   
The chip data is in the GRCh37 reference, while imputed data is in the GRCh37 reference. We will lift over the chip data by liftover_plink_beds pipeline: https://github.com/dnanexus-rnd/liftover_plink_beds\n". The protocol is described in the file `run_liftover.ipynb`. 

## Preprocess genotype data
   
### Chip data

#### Merge chromosome data  

In [ ]:
run_merge="cp ${geno_in} . ;\
           ls *.bed | sed -e 's/.bed//g' > files_to_merge.txt;\
           plink --merge-list files_to_merge.txt \
                 --make-bed \
                 --chr 1-22 \
                 --allow-extra-chr \
                 --out ${bed_base};\
           rm files_to_merge.txt;"
             
dx run swiss-army-knife \
           -icmd="${run_merge}" \
           --tag="Step0" \
           --instance-type ${instance} \
           --destination="${geno_file_dir}/" \
           --brief --yes --wait


job-Gb74yZQJFFQbp5xgzJxgzg3q


#### Filter genotypes

In [2]:
run_plink_qc="plink2 --bfile ${bed_base} \
                     --keep ${pheno_in}  \
                     --autosome \
                     --maf 0.01 --mac 20 --geno 0.1 \
                     --hwe 1e-15 --mind 0.1 \
                     --write-snplist \
                     --write-samples \
                     --no-id-header \
                     --out geno_qc_pass"

dx run swiss-army-knife \
   -iin="${geno_file_dir}/${bed_base}.bed" \
   -iin="${geno_file_dir}/${bed_base}.bim" \
   -iin="${geno_file_dir}/${bed_base}.fam" \
   -iin="${pheno_file_dir}/${pheno_in}" \
   -icmd="${run_plink_qc}" \
   --tag="Step0" \
   --instance-type ${instance} \
   --destination="${geno_file_dir}/" \
   --brief --yes --wait
   

job-Gb952BjJFFQbv07q75Kxv3G2


## Run Regenie

We will run with similar settings as in FinnGen.

### Regenie step 1: Fit model

In [ ]:
run_regenie_step1="regenie \
  --step 1 \
  --bt \
  --bed ${bed_base} \
  --extract geno_qc_pass.snplist \
  --phenoFile ${pheno_in} \
  --phenoColList '${pheno_cols}' \
  --covarFile ${pheno_in} \
  --covarColList '${covar_cols}' \
  --bsize 1000 \
  --lowmem \
  --gz \
  --threads 16 \
  --out ${name}_step1"

dx run swiss-army-knife \
   -iin="${geno_file_dir}/${bed_base}.bed" \
   -iin="${geno_file_dir}/${bed_base}.bim" \
   -iin="${geno_file_dir}/${bed_base}.fam" \
   -iin="${geno_file_dir}/geno_qc_pass.snplist" \
   -iin="${pheno_file_dir}/${pheno_in}" \
   -icmd="${run_regenie_step1}" \
   --tag="Step1" --instance-type ${instance} \
   --destination="${regenie_out_dir}/" \
   --brief --yes --wait


job-Gb95PzQJFFQvQbjpBy194G96


### Regenie step 2: Association testing

We perform association testing for SNP data.

In [ ]:
for chr in {1..22}; do

    run_regenie_step2="regenie \
      --step 2 \
      --bgen ${geno2_var}_c${chr}_b0_v1.bgen \
      --sample ${geno2_var}_c${chr}_b0_v1.sample \
      --ref-first \
      --phenoFile ${pheno_in} \
      --phenoColList '${pheno_cols}' \
      --covarFile ${pheno_in} \
      --covarColList '${covar_cols}' \
      --pred ${name}_step1_pred.list \
      --bt \
      --firth  \
      --firth-se\
      --approx \
      --bsize 400 \
      --pThresh 0.05 \
      --threads 16 \
      --gz \
      --out assoc_c${chr}"

    dx run swiss-army-knife \
       -iin="${geno2_indir}/${geno2_var}_c${chr}_b0_v1.bgen" \
       -iin="${geno2_indir}/${geno2_var}_c${chr}_b0_v1.sample" \
       -iin="${geno_file_dir}/geno_qc_pass.snplist" \
       -iin="${pheno_file_dir}/${pheno_in}" \
       -iin="${regenie_out_dir}/${name}_step1_pred.list" \
       -iin="${regenie_out_dir}/${name}_step1_1.loco.gz" \
       -iin="${regenie_out_dir}/${name}_step1_2.loco.gz" \
       -iin="${regenie_out_dir}/${name}_step1_3.loco.gz" \
       -icmd="${run_regenie_step2}" \
       --tag="Step2" --instance-type ${instance} \
       --destination="${regenie_out_dir}/" \
       --brief --yes

done




In [ ]:
#Wait until all chromosomes have been run 
# for chr in {1..22}; do
#   until [ -f /mnt/project${regenie_out_dir}/assoc_c${chr}.log ]
#   do
#     sleep 120s  #Check after every 2 min
#  done  
#done
#sleep 120s #Wait 2 min... log f

## Post processing

### Merge

In [9]:
#Perhaps if I make script out of this, I could make this work with SAK..

dx download -f ${regenie_out_dir}/assoc_c*.regenie.gz
for pheno in ${pheno_cols//,/ }; do
    results_out="assoc.${pheno}.regenie"
    #Title
    zcat  assoc_c*_${pheno}.regenie.gz | head -1 | sed 's/ /\t/g' > ${results_out}
    #Merge
    for chr in {1..22}; do
        zcat assoc_c${chr}_${pheno}.regenie.gz | tail -n+2 | sed 's/ /\t/g' >>  ${results_out}
    done
    #Calculate P from LOG10P column, remove values without P and INFO<0.3
    R -e "df<-as.data.frame(data.table::fread(\"${results_out}\"));\
          df['P']<-10^(-df['LOG10P']);\
          df<-subset(df,LOG10P!='NA');\
          df<-subset(df,INFO>0.3);\
          data.table::fwrite(df,paste0(\"${results_out}\",\".gz\"), sep='\t', na='NA', quote=F)"
    dx upload "${results_out}.gz" --path "${regenie_out_dir}/${results_out}.gz"

done   
rm assoc*regenie*

[===========================================================>] Completed 117,307,715 of 117,307,715 bytes (100%) /opt/notebooks/assoc_c3_resht_fg.regenie.gzz
[===========================================================>] Completed 129,313,601 of 129,313,601 bytes (100%) /opt/notebooks/assoc_c3_resht_both.regenie.gzz
[===========================================================>] Completed 127,883,693 of 127,883,693 bytes (100%) /opt/notebooks/assoc_c3_resht_ukb.regenie.gzz
[===========================================================>] Completed 153,116,496 of 153,116,496 bytes (100%) /opt/notebooks/assoc_c2_resht_ukb.regenie.gzz
[===========================================================>] Completed 140,331,833 of 140,331,833 bytes (100%) /opt/notebooks/assoc_c2_resht_fg.regenie.gzz
[===========================================================>] Completed 154,806,847 of 154,806,847 bytes (100%) /opt/notebooks/assoc_c2_resht_both.regenie.gzz
[=============================================

### Draw manhattan and qq plots

In [ ]:
#This could be run with SAK

dx download -f scripts/draw_manhattan.R
dx download -f ${regenie_out_dir}/assoc.*.regenie.gz
#Draw manhattan plots
for pheno in ${pheno_cols//,/ }; do
   Rscript draw_manhattan.R assoc.${pheno}.regenie.gz ${pheno}
done
dx upload *.png --path ${regenie_out_dir}/


[===========================================================>] Completed 834 of 834 bytes (100%) /opt/notebooks/draw_manhattan.R.R
[===========================================================>] Completed 2,267,139,554 of 2,267,139,554 bytes (100%) /opt/notebooks/assoc.resht_both.regenie.gzz
[===========================================================>] Completed 2,242,810,892 of 2,242,810,892 bytes (100%) /opt/notebooks/assoc.resht_ukb.regenie.gzz
[===========================================================>] Completed 2,062,552,170 of 2,062,552,170 bytes (100%) /opt/notebooks/assoc.resht_fg.regenie.gzz
Loading required package: data.table
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: qqman

Fo